# レポート課題

以下の問題について，この `report.ipynb` を編集し，完成した自分が編集した .ipynb ファイルをGoogle Classroomの該当課題に提出してください．（もしこのノートブックをファイルに保存したいときには，メニューの「file」→「Download」で .ipynb ファイルをダウンロード出来るので，それを提出してください．）


- 適宜コメントを入れてください．
- 提出する際のファイル名は，「`学籍番号.ipynb`」としてください．
- 問題1について，Markdownで書きにくい場合には，ClassroomにPDFファイルを追加して提出してもかまいません．
- 発展課題は任意とします．
- Markdownを書くときには，[Wiki](https://ja.wikipedia.org/wiki/Markdown)にあるぐらいの基本的なマークアップで書いてくれればよいです（もちろん，自分で調べてもっと高度なMarkdownを書いてくれても見られればよいです）．

# 問題1 - LALR構文解析についてまとめる（第2回の内容から）

Yaccは，***LALR構文解析*** を行う構文解析器を出力するということを第2回で説明しました．

LALR構文解析について，以下の用語などを交えて説明してください．
（すべてが含まれている必要はないですが，自分でちゃんと理解したうえでまとめること）

- 上向き構文解析
- スタック
- 構文解析表
- Shift / Reduce
- shift/reduce conflict の例
- reduce/reduce conflictの例
- 第1回の問題5 （右再帰と左再帰で，どちらがYaccにとって良いか．）

ヒント1: 書籍が沢山出ているので，それを参考にしてください．（僕らのときは，）

> 『情報科学こんせぷつ8 コンパイラの仕組み』渡邊 坦(著) ISBN：978-4-254-12708-9

が教科書に指定されていました．

ヒント2: 今回のノートブックが裏で使っているBisonのページは次のとおり，
[The Bison Parser Algorithm](https://www.gnu.org/software/bison/manual/html_node/Algorithm.html)

LALR構文解析とは、構文解析器がソースコードを解析する際に用いる手法の一つである。

## LR構文解析法

LALR構文解析は、LR構文解析法と呼ばれる方法を用いている。
この方法では、入力文字列を最初から最後に向かって読んでいき、最も右のものから構文規則に当てはめて導出していく。
LR構文解析法は、完全に人の手で構築するのが難しい反面複雑な文法に対応しやすいことから、効率良い構文解析器を機械的に生成する際によく利用されている。

### 上向き構文解析

先述したようにLR構文解析法は入力文字列の最初から開始し、終端記号から徐々に構文規則を当てはめていく。
そして、徐々に構文木を構築しながら、最終的に開始記号に到達した際に構文規則が適用できれば解析が完了したことになる。
このような手法は上向き構文解析と呼ばれている。

## LR構文解析法とスタック

LR構文解析法の特徴を自動的に実現するために欠かせないデータ構造がスタックである。
スタックを用いると、次のような操作をすることで自然にLR構文解析ができる。

- 入力記号をスタックの最上部にプッシュし次の状態へ遷移する。(Shift)
- スタックの最上部の項目を取り出し、構文規則を適用し一つの非終端記号へ還元する。(Reduce)

入力文字列の最初から開始し終端から構文規則を当てはめていく際にLIFOなデータ構造であるスタックは大変適しているといえる。

## 構文解析表

上で述べたスタックの操作を具体的に定義したものが構文解析表である。
この表は、事前に定めた構文規則を元に生成される。
実際に構文解析を行う際、現在の状態と次に入力された記号の2つを用いて表を参照することで、
次にShift / Reduceのどちらを行うかや、Reduceする場合に適用する操作を確定させることができる。

### LALR構文解析法

LALR構文解析法は、LR構文構文解析法をより効率的に行うためのものものである。
具体的には、次に入力された記号に加えて先の記号も読むことで、とりうる状態の数を削減し構文解析表を小さくしている。

### Shift/Reduce Conflict

構文解析表において、現在の状態と入力された記号をもとに次の操作を導出する際、構文規則が曖昧だと次の操作が複数考えられる場合がある。
このうち、Shift操作かReduce操作か決定できない状態をShift/Reduce Conflictと呼んでいる。

たとえば

# 問題2 - 電卓の拡張（第1回の内容から）

第1回で作成した電卓を自由に拡張して，以下にコードを載せてください．
どんな機能を追加したかをMarkdownにまとめて書いて，コードには適宜コメントを入れてください．

- 授業の練習問題で自分が作成したものを含んでいてもよいです．
- Yaccの**演算子順序**について自分で調べて利用してくれてもよいです．
- 授業で対応していなかった演算・実数・関数に対応するなど，一般の関数電卓にある機能でもよいですし，自分で考えた新たな機能でもよいです（プログラムになってしまうと，次の問題3になってしまうので，あくまで電卓の範囲で）

どんな機能を追加したのか，ここに書く

In [5]:
/* lex f2.lex */
exp      "e"
digit [0-9]
white [\t ]
%%
{digit}+({exp}{digit}+)? { return NUM; } /* 符号つき実数(指数も可能) */
[+*-/%^()]       { return yytext[0]; }
"\n"      { return '\n'; }
{white}   { ; }

[Lex] flex generates lex.yy.c successfully

In [6]:
/* yacc f2.yacc  */
%token NUM;
%left '+' '-'
%left '*' '/' '%'
%right '^'
%%
exprlist:
        | exprlist expr '\n'    { printf("%d\n", $2); }
        ;
expr    : expr '+' expr     { $$ = $1 + $3; }
        | expr '-' expr     { $$ = $1 - $3; }
        | expr '*' expr     { $$ = $1 * $3; }
        | expr '/' expr     { $$ = $1 / $3; }
        | expr '%' expr     { $$ = $1 % $3; }
        | '(' expr ')'      { $$ = $2; }
        | NUM               { $$ = atoi(yytext); }
        ;
%%

[Yacc] bison generates y.tab.c successfully

In [7]:
/* c f2.c */
#include <stdio.h>
#include <stdlib.h>
extern char *yytext;

#include "y.tab.c"
#include "lex.yy.c"

int main() {
  yyparse();
  return 0;
}

[C] gcc generates a.out successfully

In [8]:
/* a.out */
3 + 2 * 4 % 3

5


# 問題3 - 自分のプログラミング言語の拡張（第2・3回の内容から）

第2・3回で作成したプログラミング言語を自由に拡張して，以下にコードを載せてください．どんな機能を追加したか（複数推奨）をMarkdownでまとめて書いて，コードには適宜コメントを入れてください．

- 授業の練習問題で自分が作成したものを含んでもよいです．
- インタプリタでもコンパイラでもどちらでもよいです．
  - コンパイラの場合，出力先の言語は授業でやったC・x64アセンブリ言語のどちらかにしてください（出力先を変えてみるのは，以下で任意発展課題としています）
- たとえば，以下のような拡張が考えられます．（もちろん，これ以外の拡張もどんどん入れてください）
  - `if`文
  - `for`文
  - `switch`文
  - サブルーチン（値を返さない）
  - 関数（値を返す）
    - アセンブリ言語でやるとしたら，ラベルなどを付けてジャンプすることになるはず．どうスタックを使うかなどを考えないといけない．
  - 配列（1次元でよい）

## ユーティリティ関連

### println関数の追加
println関数を追加し、末尾改行付きで文字列を出力することができるようにした。
ループ内でprintを行う場合はこういう関数があると便利である。

### true / falseの追加
常に真 / 偽を表すtrue / false句の追加を行った。
これと後述するif文・break文を組み合わせることで、条件分岐やforループ相当のwhile文を書くことができる。

### 条件分岐関連

#### if文の追加
if (条件) {処理} の形式で条件分岐を行うif文を追加した。
else句については、shift / reduce conflictを解消できなかったため追加を断念した。

#### break句の追加
break; の形式でC言語のbreak;を出力するようにした。
これにより、while文などを明示的に脱出できるようになった。

In [382]:
/* lex f3.lex */
alpha   [a-zA-Z]
digit   [0-9]
white   [\n\t ]
%%
read                        { return READ; }
print                       { return PRINT; }
println                     { return PRINTLN; }
while                       { return WHILE; }
if                          { return IF; }
break                       { return BREAK; }
true                        { return TRUE; }
false                       { return FALSE; }
"++"                        { return PLUSPLUS; }
"<="                        { return LTE; }
">="                        { return GTE; }
"=="                        { return EQ; }
"&&"                        { return AND; }
"||"                        { return OR; }
{alpha}({alpha}|{digit})*   { return IDENT; }
{digit}+                    { return NUM; }
[+\-*/%=!();{}<>]               { return yytext[0]; }
{white}

[Lex] flex generates lex.yy.c successfully

In [383]:
/* yacc f3.yacc --debug */
%union {
  Node* np;
  int i;
}
%type <np> stlist stat expr prim var cond
%token NUM IDENT READ PRINT PRINTLN WHILE IF BREAK
%token PLUSPLUS LTE GTE EQ AND OR TRUE FALSE


%left OR
%left AND
%left '<' '>' LTE GTE EQ
%right '!'
%%
prog   : IDENT '{' stlist '}'       { emit_c($3); return 0; }
       ;
stlist :                            { $$ = NULL; }
       | stlist stat                { $$ = createNode(T_STLIST, $1, $2); }
       ;
stat   : var '=' expr ';'           { $$ = createNode(T_ASSIGN, $1, $3); }
       | READ var ';'               { $$ = createNode(T_READ, $2, NULL); }
       | PRINT expr ';'             { $$ = createNode(T_PRINT, $2, NULL); }
       | PRINTLN expr ';'           { $$ = createNode(T_PRINTLN, $2, NULL); }
       | IF '(' cond ')' stat       { $$ = createNode(T_IF, $3, $5); }
       | WHILE '(' cond ')' stat    { $$ = createNode(T_WHILE, $3, $5); }
       | BREAK ';'                  { $$ = createNode(T_BREAK, NULL, NULL); }
       | '{' stlist '}'             { $$ = createNode(T_BLOCK, $2, NULL); }
       ;
expr   : prim                       { $$ = $1; }
       | expr '+' prim              { $$ = createNode(T_ADD, $1, $3); }
       | expr '-' prim              { $$ = createNode(T_SUB, $1, $3); }
       | expr '*' prim              { $$ = createNode(T_MUL, $1, $3); }
       | expr '/' prim              { $$ = createNode(T_DIV, $1, $3); }
       | expr '%' prim              { $$ = createNode(T_MOD, $1, $3); }
       ;
prim   : NUM                        { $$ = createNode(T_NUM, atoi(yytext), NULL); }
       | var                        { $$ = createNode(T_VAR, $1, NULL); }
       | '(' expr ')'               { $$ = $2; }
       | var PLUSPLUS               { $$ = createNode(T_PP, $1, NULL); }
       ;
var    : IDENT                      { $$ = lookup(yytext); }
       ;
cond   : expr '<' expr     { $$ = createNode(T_LT, $1, $3); }
       | expr '>' expr     { $$ = createNode(T_GT, $1, $3); }
       | expr GTE expr     { $$ = createNode(T_GTE, $1, $3); }
       | expr LTE expr     { $$ = createNode(T_LTE, $1, $3); }
       | expr EQ expr      { $$ = createNode(T_EQ, $1, $3); }
       | cond AND cond     { $$ = createNode(T_AND, $1, $3); }
       | cond OR cond      { $$ = createNode(T_OR, $1, $3); }
       | '!' '(' cond ')'  { $$ = createNode(T_NOT, $3, NULL); }
       | TRUE             { $$ = createNode(T_TRUE, NULL, NULL); }
       | FALSE            { $$ = createNode(T_FALSE, NULL, NULL); }
       ;


[Yacc] bison generates y.tab.c successfully

In [384]:
/* c f3.c */
#include <stdio.h>
struct stab {
  int val;
  char name[20];
} stab[100];
int stabuse = 0;

extern char *yytext;

#define T_STLIST 1
#define T_ASSIGN 2
#define T_READ   3
#define T_PRINT  4
#define T_ADD    5
#define T_SUB    6
#define T_NUM    7
#define T_VAR    8
#define T_WHILE  9
#define T_LT     10
#define T_GT     11
#define T_LTE    12
#define T_GTE    13
#define T_EQ     14
#define T_NOT    15
#define T_AND    16
#define T_OR     17
#define T_PP     18
#define T_BLOCK  19
#define T_IF    20
#define T_BREAK 21
#define T_TRUE  22
#define T_FALSE 23
#define T_MUL   24
#define T_DIV   25
#define T_MOD   26
#define T_PRINTLN 27

typedef struct Node {
    int node_type;
    struct Node* left;
    struct Node* right;
} Node;


// 引数にとる文字列が表にあれば見つかった位置を返して，なければ追加する関数
int lookup(char *s) {
  int i;
  // 表の使っているところを順に見ていって
  for (i = 0; i < stabuse; ++i) {
    if (strcmp(stab[i].name, s) == 0) {
      // 見つかった場合はその位置を返す
      return i;
    }
  }
  if (stabuse >= 99) {
    // 変数の上限の個数を超えてしまった
    printf("table overflow,\n");
    exit(1);
  }

  // 表になかったので追加する
  strcpy(stab[stabuse].name, s);
  return stabuse++;  // 追加した位置を返す
}

// 新しいノードを作る関数
Node* createNode(int node_type, Node* left, Node* right) {
    Node* newNode = (Node*)malloc(sizeof(Node));
    newNode->node_type = node_type;
    newNode->left = left;
    newNode->right = right;
    return newNode;
}

void _emit_c(Node* node) {
  if (node == NULL) {
    return;
  }
  switch (node->node_type) {
    case T_STLIST:
      if (node->left != NULL) { _emit_c(node->left); }
      _emit_c(node->right);
      break;
    case T_ASSIGN:
      printf("\tv%d = ", node->left);
      _emit_c(node->right);
      printf(";\n");
      break;
    case T_READ:
      printf("\tscanf(\"%%d\", &v%d);\n", node->left);
      break;
    case T_PRINT:
      printf("\tprintf(\"%%d\",");
      _emit_c(node->left);
      printf(");\n");
      break;
    case T_PRINTLN:
      printf("\tprintf(\"%%d\",");
      _emit_c(node->left);
      printf(");\n");
      printf("\tprintf(\"\\n\");");
      printf("\n");
      break;
    case T_ADD:
      _emit_c(node->left);
      printf(" + ");
      _emit_c(node->right);
      break;
    case T_SUB:
      _emit_c(node->left);
      printf(" - ");
      _emit_c(node->right);
      break;
    case T_NUM:
      printf("%d", node->left);
      break;
    case T_VAR:
      printf("v%d", node->left);
      break;
    case T_WHILE:
      printf("\twhile (");
      _emit_c(node->left);
      printf(") {\n\t");
      _emit_c(node->right);
      printf("\t}\n");
      break;
    case T_LT:
      _emit_c(node->left);
      printf(" < ");
      _emit_c(node->right);
      break;
    case T_GT:
      _emit_c(node->left);
      printf(" > ");
      _emit_c(node->right);
      break;
    case T_LTE:
      _emit_c(node->left);
      printf(" <= ");
      _emit_c(node->right);
      break;
    case T_GTE:
      _emit_c(node->left);
      printf(" >= ");
      _emit_c(node->right);
      break;
    case T_EQ:
      _emit_c(node->left);
      printf(" == ");
      _emit_c(node->right);
      break;
    case T_NOT:
      printf("!");
      _emit_c(node->left);
      break;
    case T_AND:
      _emit_c(node->left);
      printf(" && ");
      _emit_c(node->right);
      break;
    case T_OR:
      _emit_c(node->left);
      printf(" || ");
      _emit_c(node->right);
      break;
    case T_PP:
      printf("v%d++", node->left);
      break;
    case T_BLOCK:
      _emit_c(node->left);
      break;
    case T_IF:
      printf("\tif (");
      _emit_c(node->left); /* 条件式 */
      printf(") {\n\t");
      _emit_c(node->right); /* 文 */
      printf("\t}\n");
      break;
    case T_BREAK:
      printf("\tbreak;\n");
      break;
    case T_TRUE:
      printf("1");
      break;
    case T_FALSE:
      printf("0");
      break;
    case T_MUL:
      _emit_c(node->left);
      printf(" * ");
      _emit_c(node->right);
      break;
    case T_DIV:
      _emit_c(node->left);
      printf(" / ");
      _emit_c(node->right);
      break;
    case T_MOD:
      _emit_c(node->left);
      printf(" %% ");
      _emit_c(node->right);
      break;
  }
}

void emit_c(Node* node) {
  printf("#include <stdio.h>\n");
  printf("int main() {\n");
  int i;
  for (i = 0; i < stabuse; i++) {
    printf("\tint v%d;\n", i);
  }
  _emit_c(node);
  printf("}\n");
}

#include "y.tab.c"
#include "lex.yy.c"

int main() {
  yydebug = 1;
  yyparse();
  return 0;
}

[C] gcc generates a.out successfully

In [386]:
/* a.out */
main {
  read a;
  while (true) {
    if (a >= 100) {
      break;
    }
    read b;
    a = a + b;
    println a;
  }
}

Starting parse
Entering state 0
Stack now 0
Reading a token
Next token is token IDENT ()
Shifting token IDENT ()
Entering state 1
Stack now 0 1
Reading a token
Next token is token '{' ()
Shifting token '{' ()
Entering state 3
Stack now 0 1 3
Reducing stack by rule 2 (line 17):
-> $$ = nterm stlist ()
Entering state 5
Stack now 0 1 3 5
Reading a token
Next token is token READ ()
Shifting token READ ()
Entering state 7
Stack now 0 1 3 5 7
Reading a token
Next token is token IDENT ()
Shifting token IDENT ()
Entering state 6
Stack now 0 1 3 5 7 6
Reducing stack by rule 22 (line 41):
   $1 = token IDENT ()
-> $$ = nterm var ()
Entering state 17
Stack now 0 1 3 5 7 17
Reading a token
Next token is token ';' ()
Shifting token ';' ()
Entering state 29
Stack now 0 1 3 5 7 17 29
Reducing stack by rule 5 (line 21):
   $1 = token READ ()
   $2 = nterm var ()
   $3 = token ';' ()
-> $$ = nterm stat ()
Entering state 15
Stack now 0 1 3 5 15
Reducing stack by rule 3 (line 18):
   $1 = nterm stlist ()

# 発展問題1 - コード生成（第3回の内容から）

第3回で，C言語とx64アセンブリ言語に出力するコンパイラを作成しました．それを参考にして，他のプログラミング言語に出力するコンパイラを作成してください．どんなプログラミング言語に出力するかと，必要に応じて実行モデルについてどうしたか（翻訳するときにどう決めたか）をMarkdownにまとめて書いて，コードには適宜コメントを入れてください．

- 自分の知っているどんな言語でもいいです．
  - 言語処理系論で出てきた WebAssemblyについて自分で調べてみて出力できると，おもしろいかもしれない（その場合は呼び出すJavaScriptも書いてほしい）．

どんな言語に出力したのか，ここに書く

In [ ]:
/* lex f4.lex */

In [ ]:
/* yacc f4.yacc */

In [ ]:
/* c f4.c */

In [ ]:
/* a.out */

# 発展問題2 - プログラミングではない用途（第3回の内容から）

第3回で，簡単な英文解析プログラムを作成しました．同様にして，プログラミング言語ではない用途でLex・Yaccを使用したものを自由に作成して，以下にコードを載せてください．どんなプログラムかをMarkdownでまとめて書いて，コードには適宜コメントを入れてください．

- 英文の例を単純に拡張するのでもよいです．
- JSONやXMLのようなデータを読み込むプログラムでもよいです．
- コマンドのオプションを読み込むときに使うプログラムでもよいです．
  - たとえば，`ssh`だったら（`% ssh -l ユーザー名`）のようにコマンドを書くけれど，それを解釈するプログラム．間違っていたら`man`コマンドの結果のように使い方を表示すしてみるなど

どんな機能を追加したのか，ここに書く

In [ ]:
/* lex f5.lex */

In [ ]:
/* yacc f5.yacc */

In [ ]:
/* c f5.c */

In [ ]:
/* a.out */

# 発展問題3 - 最適化

授業資料で作成したコンパイラは，抽象構文木を作成しただけでした．
抽象構文木の構造をいじってやれば，コンパイラに最適化を実装できます．この課題では，自由に考えて（調べて）自分が作成したコンパイラになんらかの最適化手法を取り入れてください．

- ここまでの問題のどのバージョンのコンパイラでもよいです．
  - 最適化の効果が見やすいので，C言語にコード生成してくれるとありがたい． 
- たとえば，
  - 別々の式になっていた足し算を1つにマージするとか（実際問題として実行速度が上がるわけではないが，抽象構文木の形を変える練習として）
  - [SSA](https://ja.wikipedia.org/wiki/%E9%9D%99%E7%9A%84%E5%8D%98%E4%B8%80%E4%BB%A3%E5%85%A5) 形式に変換してみるとか
  - ループ最適化とか

In [ ]:
/* lex f6.lex */

In [ ]:
/* yacc f6.yacc */

In [ ]:
/* c f6.c */

In [ ]:
/* a.out */